In [3]:
import os
import itertools
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import extract_v3
import categorize
import cv2

# Initialize lists for features and labels
X_baseline_angle = []
X_top_margin = []
X_letter_size = []
X_line_spacing = []
X_word_spacing = []
X_pen_pressure = []
X_slant_angle = []
y_t1 = []
y_t2 = []
y_t3 = []
y_t4 = []
y_t5 = []
y_t6 = []
y_t7 = []
y_t8 = []
page_ids = []

label_list_path = r"C:\Users\parth\OneDrive\Desktop\IBM_Work\New folder\label_list"

if os.path.isfile(label_list_path):
    print("Info: label_list found.")
    with open(label_list_path, "r") as labels:
        lines = labels.readlines()
        for i in range(0, len(lines), 2):
            feature_line = lines[i].strip().split()
            label_line = lines[i+1].strip().split()
            
            if len(feature_line) != 8 or len(label_line) != 8:
                print(f"Skipping line pair due to incorrect format: {lines[i].strip()} and {lines[i+1].strip()}")
                continue

            X_baseline_angle.append(float(feature_line[0]))
            X_top_margin.append(float(feature_line[1]))
            X_letter_size.append(float(feature_line[2]))
            X_line_spacing.append(float(feature_line[3]))
            X_word_spacing.append(float(feature_line[4]))
            X_pen_pressure.append(float(feature_line[5]))
            X_slant_angle.append(float(feature_line[6]))
            page_ids.append(feature_line[7])

            y_t1.append(float(label_line[0]))
            y_t2.append(float(label_line[1]))
            y_t3.append(float(label_line[2]))
            y_t4.append(float(label_line[3]))
            y_t5.append(float(label_line[4]))
            y_t6.append(float(label_line[5]))
            y_t7.append(float(label_line[6]))
            y_t8.append(float(label_line[7]))

    # Create datasets for each trait
    X_t1 = list(zip(X_baseline_angle, X_slant_angle))
    X_t2 = list(zip(X_letter_size, X_pen_pressure))
    X_t3 = list(zip(X_letter_size, X_top_margin))
    X_t4 = list(zip(X_line_spacing, X_word_spacing))
    X_t5 = list(zip(X_slant_angle, X_top_margin))
    X_t6 = list(zip(X_letter_size, X_line_spacing))
    X_t7 = list(zip(X_letter_size, X_word_spacing))
    X_t8 = list(zip(X_line_spacing, X_word_spacing))

    classifiers = []
    datasets = [
        (X_t1, y_t1),
        (X_t2, y_t2),
        (X_t3, y_t3),
        (X_t4, y_t4),
        (X_t5, y_t5),
        (X_t6, y_t6),
        (X_t7, y_t7),
        (X_t8, y_t8),
    ]
    random_states = [8, 16, 32, 64, 42, 52, 21, 73]

    for i, (X, y) in enumerate(datasets):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=random_states[i])
        if len(set(y_train)) > 1:  # Ensure there is more than one class
            clf = SVC(kernel='rbf')
            clf.fit(X_train, y_train)
            accuracy = accuracy_score(clf.predict(X_test), y_test)
            print(f"Classifier {i+1} accuracy: {accuracy}")
            classifiers.append(clf)
        else:
            print(f"Classifier {i+1} was not trained due to insufficient class diversity in training data.")
            classifiers.append(None)  # Append None to maintain the correct index

    # Prediction loop
    while True:
        file_name = input("Enter file name to predict or 'z' to exit: ")
        if file_name == 'z':
            break

        # Construct the full path to the image file
        image_path = os.path.join(r'C:\Users\parth\Downloads\images\images', file_name)
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Error: Image file '{file_name}' not found or could not be loaded.")
            continue

        raw_features = extract_v3.start(image_path)  # Use the correct path

        raw_baseline_angle = raw_features[0]
        baseline_angle, comment = categorize.determine_baseline_angle(raw_baseline_angle)
        print("Baseline Angle: " + comment)

        raw_top_margin = raw_features[1]
        top_margin, comment = categorize.determine_top_margin(raw_top_margin)
        print("Top Margin: " + comment)

        raw_letter_size = raw_features[2]
        letter_size, comment = categorize.determine_letter_size(raw_letter_size)
        print("Letter Size: " + comment)

        raw_line_spacing = raw_features[3]
        line_spacing, comment = categorize.determine_line_spacing(raw_line_spacing)
        print("Line Spacing: " + comment)

        raw_word_spacing = raw_features[4]
        word_spacing, comment = categorize.determine_word_spacing(raw_word_spacing)
        print("Word Spacing: " + comment)

        raw_pen_pressure = raw_features[5]
        pen_pressure, comment = categorize.determine_pen_pressure(raw_pen_pressure)
        print("Pen Pressure: " + comment)

        raw_slant_angle = raw_features[6]
        slant_angle, comment = categorize.determine_slant_angle(raw_slant_angle)
        print("Slant: " + comment)

        # Check if each classifier exists before attempting to use it
        if classifiers[0]:
            print("Emotional Stability: ", classifiers[0].predict([[baseline_angle, slant_angle]]))
        if classifiers[1]:
            print("Mental Energy or Will Power: ", classifiers[1].predict([[letter_size, pen_pressure]]))
        if classifiers[2]:
            print("Modesty: ", classifiers[2].predict([[letter_size, top_margin]]))
        if classifiers[3]:
            print("Personal Harmony and Flexibility: ", classifiers[3].predict([[line_spacing, word_spacing]]))
        if classifiers[4]:
            print("Lack of Discipline: ", classifiers[4].predict([[slant_angle, top_margin]]))
        if classifiers[5]:
            print("Poor Concentration: ", classifiers[5].predict([[letter_size, line_spacing]]))
        if classifiers[6]:
            print("Non Communicativeness: ", classifiers[6].predict([[letter_size, word_spacing]]))
        if classifiers[7]:
            print("Social Isolation: ", classifiers[7].predict([[line_spacing, word_spacing]]))
        
        print("---------------------------------------------------")
else:
    print("Error: label_list file not found.")


Info: label_list found.
Classifier 1 was not trained due to insufficient class diversity in training data.
Classifier 2 accuracy: 1.0
Classifier 3 accuracy: 1.0
Classifier 4 accuracy: 1.0
Classifier 5 accuracy: 1.0
Classifier 6 accuracy: 1.0
Classifier 7 accuracy: 1.0
Classifier 8 accuracy: 1.0


Enter file name to predict or 'z' to exit:  666-0.png


Baseline Angle: DESCENDING
Top Margin: MEDIUM OR BIGGER
Letter Size: SMALL
Line Spacing: BIG
Word Spacing: MEDIUM
Pen Pressure: LIGHT
Slant: MODERATELY INCLINED
Mental Energy or Will Power:  [1.]
Modesty:  [1.]
Personal Harmony and Flexibility:  [0.]
Lack of Discipline:  [0.]
Poor Concentration:  [0.]
Non Communicativeness:  [0.]
Social Isolation:  [1.]
---------------------------------------------------


Enter file name to predict or 'z' to exit:  z
